In [24]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 

Defining our features

In [28]:
authors = []
dates = []
statements = []
sources = []
targets = []

unique_labels = []

Scraping the Website

In [26]:
def scrape_website(pageno):
    page_num = str(pageno)
    url = 'https://www.politifact.com/factchecks/list/?page='+str(pageno)+'&'
    webpage = requests.get(url) #making a request to the website
    soup = BeautifulSoup(webpage.text,'html.parser')
    #get the location of the information
    # footer and div are tags
    statement_footer = soup.find_all('footer', attrs = {'class':'m-statement__footer'}) #author and data location
    statement_quote = soup.find_all('div',attrs={'class':'m-statement__quote'}) #statement location
    statement_meta = soup.find_all('div',attrs={'class':'m-statement__meta'}) #source location
    statement_target = soup.find_all('div',attrs={'class':'m-statement__meter'}) #Target location (score card)
    
    #loop through the statement_footer
    for i in statement_footer:
        link1 = i.text.strip()
        name_and_date = link1.split()
        first_name = name_and_date[1]
        last_name = name_and_date[2]
        full_name = first_name+' '+last_name
        month = name_and_date[4]
        day = name_and_date[5]
        year = name_and_date[6]
        date = day+' '+month+' '+year
        authors.append(full_name)
        dates.append(date)
       
    #loop through statement_quote
    for i in statement_quote:
        link2 = i.find_all('a')
        statement_text = link2[0].text.strip()
        statements.append(statement_text) 
    
    #loop through the meta
    for i in statement_meta:
        link3 = i.find_all('a')
        source_text = link3[0].text.strip()
        sources.append(source_text)
        
    #loop through the target
    for i in statement_target:
        #searching another (div) tag, find (nested)tag 'img', get 'alt' from img tag
        link4 = i.find('div',attrs={'class':'c-image'}).find('img').get('alt')
        targets.append(link4)

In [29]:
n = 34
for i in range(1,n+1):
    scrape_website(i)

#creating the dataframe
data = pd.DataFrame(columns=['author','statement','date','source','target'])
data['author'] = authors
data['statement'] = statements
data['date'] = dates
data['source'] = sources
data['target'] = targets

In [31]:
print(len(statements))

1020


In [32]:
data.to_csv("C:/Users/HP/OneDrive/Desktop/Ram/Projects/Fake News Detection/src/Article_Data.csv")

In [33]:
data.head(10)

,author,statement,date,source,target
0,Maria Briceño,Los incendios en California “han llegado a Tij...,"13, January 2025",Facebook posts,false
1,Jeff Cercone,Video shows people in Los Angeles looting from...,"13, January 2025",Instagram posts,pants-fire
2,Sofia Ahmed,A photo shows a Christian’s home that “miracul...,"13, January 2025",Facebook posts,false
3,Ciara O'Rourke,Image shows former President Barack Obama was ...,"13, January 2025",Threads posts,pants-fire
4,Maria Briceño,Esta imagen muestra los incendios en Californi...,"13, January 2025",Facebook posts,false
5,Madison Czopek,“Blue items that survive” California wildfires...,"10, January 2025",Threads posts,false
6,Maria Briceño,Imagen muestra el letrero de Hollywood en Cali...,"10, January 2025",Facebook posts,false
7,Loreben Tuquero,Video shows a man saving a bunny during the 20...,"10, January 2025",Facebook posts,false
8,Loreben Tuquero,Image shows the Hollywood sign was on fire as ...,"10, January 2025",Social Media,false
9,Ciara O'Rourke,Video shows Dr. Mehmet Oz and rapper Snoop Dog...,"10, January 2025",Viral image,false


In [35]:
unique_labels = list(set(data['target']))

In [36]:
print(unique_labels)

['mostly-true', 'true', 'false', 'barely-true', 'half-true', 'full-flop', 'pants-fire']
